In [ ]:
'''
Author: Kazeem Oladejo
Last updated: 07-Mar-2023
Description: SBERT Semantic Search experiment
'''

In [ ]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 4.6 MB/s 
     |████████████████████████████████| 5.8 MB 53.6 MB/s 
     |████████████████████████████████| 1.3 MB 52.7 MB/s 
     |████████████████████████████████| 182 kB 74.9 MB/s 
     |████████████████████████████████| 7.6 MB 47.8 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=4faf8bee7c14f19d1c59ac8e8b92c14f030204426fa1412a92a612cd6f846151
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers


In [ ]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import re
import time

In [ ]:
# from google.colab import files
# import io
from google.colab import drive 
drive.mount("/content/drive", force_remount=True)

# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_CleanTweets = pd.read_csv('gdrive/<data file>.csv', dtype=str)
df_CleanTweets.head()

Available fields:<br>
TID<br>
OrigTweet<br>
InReplyTo<br>
ArticleTitle<br>
CountReplyTweets<br>
CountReplyWords<br>
CountReplyChars<br>
CleanTweet<br>

In [ ]:
len(daf_CleanTweets)

148836

In [ ]:
df_GTrInput = pd.read_csv('/content/drive/MyDrive/GroundTruthDecomposed.csv', dtype=str)

<b>Available fields:</b><br>
TID<br>
OrigTweet<br>
CleanTweetNoHttp	<br>
CleanTweetNoSpecChar	<br>
HashTags	<br>
Mentions	<br>
InReplyTo	<br>
ArticleTitle	<br>
CountReplyTweets	<br>
CountReplyWords	<br>
CountReplyChars	<br>
Target	<br>
Label

In [ ]:
df_GTrInput.head()

In [ ]:
df_reGTr = pd.DataFrame()

In [ ]:
df_reGTr['TID'] = df_GTrInput['TID']
df_reGTr['OrigTweet'] = df_GTrInput['OrigTweet']
df_reGTr['InReplyTo'] = df_GTrInput['InReplyTo']
df_reGTr['ArticleTitle'] = df_GTrInput['ArticleTitle']
df_reGTr['CountReplyTweets'] = df_GTrInput['CountReplyTweets']
df_reGTr['CountReplyWords'] = df_GTrInput['CountReplyWords']
df_reGTr['CountReplyChars'] = df_GTrInput['CountReplyChars']
df_reGTr['CleanTweet'] = df_GTrInput['CleanTweetNoSpecChar']


In [ ]:
df_CleanTweets = pd.concat([df_CleanTweets,df_reGTr])

In [ ]:
len(df_CleanTweets)

273813

In [ ]:
# file generated by JatoDR (Deep Reader)
df_DeepRead = pd.read_csv('/content/drive/MyDrive/CGT_DeepRead_top20pct_max20.csv')
df_DeepRead.head()

<b>Available fields:</b><br>
TID	<br>
ArticleTitle	<br>
CitedTweet	<br>
Target	<br>
Label	<br>
TopicID	<br>
TopicScore

In [ ]:
len(df_DeepRead)

162

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2', device='cuda')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
rangeDR = len(df_DeepRead)
for i in range(rangeDR):
  # define deep read item query embedding
  iQID = str(df_DeepRead['TID'].loc[i])
  iQuery = [df_DeepRead['CitedTweet'].loc[i]]
  query_embedding = model.encode(iQuery, convert_to_tensor=True)

  print('Finding semantic matches for '+str(iQID)+' iter '+str(i+1) + ' of '+str(rangeDR))

  # define children embeddings (use ten most populated replies to iQID)
  df_children = df_CleanTweets[(df_CleanTweets.InReplyTo==iQID)]
  df_children = df_children.sort_values(['CountReplyWords'], ascending=(False))
  df_children = df_children.head(10)

  # define corpus embeddings
  df_corpus = df_CleanTweets[(df_CleanTweets.InReplyTo!=iQID)]
  corpus = list(df_corpus['CleanTweet'])
  corpus_embeddings = model.encode(corpus, convert_to_tensor=True)

  # generate up to k matches where k is 260 or len(corpus) if less than 260
  k = min(260, len(corpus))

  # run semantic search for iQID
  iQID_results = util.semantic_search(query_embedding, corpus_embeddings, top_k=k)

  # write results to dataframe for iQID
  df_result = pd.DataFrame(columns=["RID","QID", "cQID", "TID", "Tweet","Score"])
  for iQID_result in iQID_results:
      for iQID_res_row in iQID_result:
        RID = iQID_res_row['corpus_id']
        score = iQID_res_row['score']

        # write output for iQID
        df_result.loc[len(df_result)] = [RID,iQID,"",df_corpus.iloc[int(RID)]['TID'],corpus[int(RID)], score]
  
  # get results for the children of iQID
  range_cQID = len(df_children)
  for c in range(range_cQID):

    cQID = str(df_children['TID'].iloc[c])
    cQuery = [df_children['CleanTweet'].iloc[c]]
    cQuery_embedding = model.encode(cQuery, convert_to_tensor=True)

    print('...finding semantic matches for '+str(cQID)+' iter '+str(c+1) + ' of '+str(range_cQID))

    # run semantic search for cQID
    cQID_results = util.semantic_search(cQuery_embedding, corpus_embeddings, top_k=k)

    # write results to dataframe for cQID
    for cQID_result in cQID_results:
      for cQID_res_row in cQID_result:
        RID = cQID_res_row['corpus_id']
        score = cQID_res_row['score']

        # write output for iQID
        df_result.loc[len(df_result)] = [RID,iQID,cQID,df_corpus.iloc[int(RID)]['TID'],corpus[int(RID)], score]

  # save to csv file   MOVE THIS UP AND MAKE FOR CQID
  filename = '/content/drive/MyDrive/'+str(i)+"_"+iQID+'.csv'
  df_result.to_csv(filename)

Finding semantic matches for 1215847283381755914 iter 135 of 162
...finding semantic matches for 1216252649395359744 iter 1 of 7
...finding semantic matches for 1215898213070393344 iter 2 of 7
...finding semantic matches for 1215887434434732033 iter 3 of 7
...finding semantic matches for 1215890430396653568 iter 4 of 7
...finding semantic matches for 1215976887580798977 iter 5 of 7
...finding semantic matches for 1215962290303926272 iter 6 of 7
...finding semantic matches for 1215994610549428225 iter 7 of 7
Finding semantic matches for 1216305661660692480 iter 136 of 162
...finding semantic matches for 1216317131161096192 iter 1 of 9
...finding semantic matches for 1216330480225746944 iter 2 of 9
...finding semantic matches for 1216364628818759685 iter 3 of 9
...finding semantic matches for 1216321140877955072 iter 4 of 9
...finding semantic matches for 1216434904407273472 iter 5 of 9
...finding semantic matches for 1216528837326057472 iter 6 of 9
...finding semantic matches for 121631